<a href="https://colab.research.google.com/github/devamsheth0806/Skin-lesion-detection/blob/master/Skin_lesion_classification_5d2ad6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'skin-cancer-mnist-ham10000:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F54339%2F104884%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240715%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240715T232556Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6fbf8c2a10ff386a13475bae6c2db4d13f89d142bbfa616558430f651c94f2a9c5394446f3011fa8ea6f0141dbd61bc53768835d223b78f0a355953e8c1b07ca636924bbd95d0164ed0e97d29d1957e196ad096c7e615e0ab2c9afb93a943aeea777864ab6b273c5893e12c40a4a97876f8cf8f17a98374839f74f78729ecd093bf6c7fb469efbc9a8dde4eb41efa326f1c66a4f5f3bb81c1cba26bad0a8a81566e4b25999d28c2212e8c48d30ba404c8319e4ab5e621ece6cbf6412da56f4c7e4b43533bdd232ec951ae40ee665272ecd5ffb09361cf957b821438287a0222a311d21a849cd5e90892685a4eaa817686d896d5162531f8960b95e45f764ddce'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Skin Lesion classification using deep learning

## Importing libraries

In [ ]:
# Graph plotting

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image


# data manipulation
import numpy as np
import pandas as pd
import os
from glob import glob
import shutil
import gc

np.random.seed(23)

# sci-kit learn tools
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

#deep learning models and tools
import keras
import tensorflow as tf
from keras import layers
from keras.utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential
from keras.layers import BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input
from tensorflow.keras.applications import EfficientNetB2
from keras import backend as K
# from keras.layers.Normalization import
from keras.optimizers import Adam, Adamax
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

## Importing dataset

In [ ]:
base_dir = os.path.join('..', 'input','skin-cancer-mnist-ham10000') # Merging images from both folders into one dictionary

imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_dir, '*', '*.jpg'))}

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [ ]:
meta_df = pd.read_csv(os.path.join(base_dir,'HAM10000_metadata.csv'))
meta_df['path'] = meta_df['image_id'].map(imageid_path_dict.get)
meta_df['cell_type'] = meta_df['dx'].map(lesion_type_dict.get)
meta_df['cell_type_idx'] = pd.Categorical(meta_df['cell_type']).codes
meta_df.head()

## Pre-processing

In [ ]:
# checking nulls
meta_df.isnull().sum()

In [ ]:
meta_df['age'].fillna(meta_df.groupby("dx")['age'].transform(lambda x: x.fillna(x.mean())), inplace=True)

In [ ]:
meta_df.isnull().sum()

In [ ]:
meta_df.dtypes

## Exploratory Data Analysis

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize= (10, 5))
meta_df['cell_type'].value_counts().plot(kind='bar', ax=ax1)

In [ ]:
meta_df['dx_type'].value_counts().plot(kind='bar')

In [ ]:
meta_df['localization'].value_counts().plot(kind='bar')

In [ ]:
meta_df['age'].hist(bins=40)

In [ ]:
meta_df['sex'].value_counts().plot(kind='bar')

In [ ]:
sns.boxplot(data=meta_df,y='age',x='cell_type_idx')

In [ ]:
fig, ax = plt.subplots(figsize=[20,5])
sns.boxplot(data=meta_df,y='age',x='cell_type')

## Loading of images

In [ ]:
t_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_index, test_index = next(t_split.split(meta_df['path'], meta_df['dx']))
x_train, x_test_meta = meta_df['path'].iloc[train_index], meta_df['path'].iloc[test_index]
y_train, y_test_meta = meta_df['dx'].iloc[train_index], meta_df['dx'].iloc[test_index]

v_split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=25)
train_index, val_index = next(v_split.split(x_train, y_train))
x_train, x_val_meta = x_train.iloc[train_index], x_train.iloc[val_index]
y_train, y_val_meta = y_train.iloc[train_index], y_train.iloc[val_index]


In [ ]:
if(os.path.exists('aug_dir')):
    shutil.rmtree('aug_dir')
if(os.path.exists('base_dir')):
    shutil.rmtree('base_dir')

In [ ]:
class_list = ['mel','bkl','bcc','akiec','vasc','df', 'nv']
train_dir = os.path.join('base_dir','train_dir')
val_dir = os.path.join('base_dir','validate_dir')
test_dir = os.path.join('base_dir','test_dir')

In [ ]:
os.makedirs(train_dir)
os.makedirs(val_dir)
os.makedirs(test_dir)


for cat in class_list:
    train_data_dir = os.path.join(train_dir, cat)
    os.mkdir(train_data_dir)
    val_data_dir = os.path.join(val_dir, cat)
    os.mkdir(val_data_dir)
    test_data_dir = os.path.join(test_dir, cat)
    os.mkdir(test_data_dir)


def categorical_storing(row, dataset_type):
    label = row['dx']
    # source path to image
    src = row['path']
    fname = os.path.basename(src)
    if dataset_type == 'train':
        dst = os.path.join(train_dir, label, fname)
        # copy the image from the source to the destination
        _ = shutil.copyfile(src, dst)
    elif dataset_type == 'test':
        dst = os.path.join(test_dir, label, fname)
        # copy the image from the source to the destination
        _ = shutil.copyfile(src, dst)
    else:
        dst = os.path.join(val_dir, label, fname)
        # copy the image from the source to the destination
        _ = shutil.copyfile(src, dst)

In [ ]:
pd.concat([x_train, y_train], axis=1).apply(lambda row: categorical_storing(row, dataset_type='train'),axis=1)
pd.concat([x_val_meta, y_val_meta], axis=1).apply(lambda row: categorical_storing(row, dataset_type='val'),axis=1)
pd.concat([x_test_meta, y_test_meta], axis=1).apply(lambda row: categorical_storing(row, dataset_type='test'),axis=1)

In [ ]:
def read_image(img_name,path):
#     print(img_name)
    image = tf.io.read_file(os.path.join(path,img_name))
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [150, 200])
    image = tf.image.per_image_standardization(image)
    return image
def save_images(row, path):
    global i
#     print(row)
    im =tf.io.encode_jpeg(tf.image.convert_image_dtype(row['images'], dtype=tf.uint8))
    tf.io.write_file(os.path.join(path,f'augmented_{i}.jpg'),im)
    i+=1
def batch_generator(df, num_batch):
    batches = np.array_split(np.arange(len(df)), num_batch)
    for batch in batches:
        yield df.iloc[batch]

In [ ]:
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.5),
    layers.RandomZoom(0.3),
    layers.RandomTranslation(0.2,0.2)
])
batch_size = 100

if(os.path.exists('aug_dir')):
    shutil.rmtree('aug_dir')
aug_dir = 'aug_dir'
os.mkdir(aug_dir)
i = 0

for img_class in class_list:
    i=0
    src_path = os.path.join(train_dir,img_class)
    save_path = os.path.join(aug_dir,img_class)
    os.mkdir(save_path)
    images_df = pd.DataFrame({'path': os.listdir(src_path)})
#     images_df = images_df.map(lambda x: read_image(x,path=src_path))
    images_df = pd.DataFrame({'images':map(lambda x: np.asarray(Image.open(os.path.join(src_path,x)).resize((150,200)))/255, images_df['path'])})
#     images_df = tf.data.Dataset.from_tensor_slices(images_df.to_dict('list'))
#     images_df = images_df.shuffle(1000)
#     images_df = images_df.batch(batch_size)
    num_aug_images_wanted = 6000
    num_files = len(os.listdir(src_path))
    print(num_files)
    num_batches = int(np.ceil((num_aug_images_wanted-num_files)/batch_size))
    print(num_batches)
#     print(save_path)
#     print(src_path)
    print("transforming:",img_class)
    for batch in batch_generator(images_df,num_batches):
        num_images = 0
        while num_images < batch_size:
            images_df = batch.map(lambda x: (data_augmentation(x, training=True)))
            num_images += len(images_df)
            images_df.iloc[:batch_size].apply(lambda row: save_images(row, path = save_path),axis=1)

In [ ]:
for img_class in class_list:
    for fname in os.listdir(os.path.join(aug_dir, img_class)):
        src = os.path.join(aug_dir, img_class,fname)
        dst = os.path.join(train_dir, img_class, fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [ ]:
print(len(os.listdir('base_dir/train_dir/nv')))
print(len(os.listdir('base_dir/train_dir/mel')))
print(len(os.listdir('base_dir/train_dir/bkl')))
print(len(os.listdir('base_dir/train_dir/bcc')))
print(len(os.listdir('base_dir/train_dir/akiec')))
print(len(os.listdir('base_dir/train_dir/vasc')))
print(len(os.listdir('base_dir/train_dir/df')))

In [ ]:
print(len(os.listdir('aug_dir/nv')))
print(len(os.listdir('aug_dir/mel')))
print(len(os.listdir('aug_dir/bkl')))
print(len(os.listdir('aug_dir/bcc')))
print(len(os.listdir('aug_dir/akiec')))
print(len(os.listdir('aug_dir/vasc')))
print(len(os.listdir('aug_dir/df')))

In [ ]:
data_df = pd.DataFrame(columns=['images','dx'])
for img_class in class_list:
    path = os.path.join(train_dir,img_class)
    files = os.listdir(path)
    temp_df = pd.DataFrame({'images':list(map(lambda x: np.asarray(Image.open(os.path.join(path,x)).resize((60,80)))/255.0,files)), 'dx':[img_class]*len(files)})
    data_df = pd.concat([data_df, temp_df])

In [ ]:
val_df = pd.DataFrame(columns=['images','dx'])
for img_class in class_list:
    path = os.path.join(val_dir,img_class)
    files = os.listdir(path)
    temp_df = pd.DataFrame({'images':list(map(lambda x: np.asarray(Image.open(os.path.join(path,x)).resize((60,80)))/255.0,files)), 'dx':[img_class]*len(files)})
    val_df = pd.concat([val_df, temp_df])

In [ ]:
test_df = pd.DataFrame(columns=['images','dx'])
for img_class in class_list:
    path = os.path.join(test_dir,img_class)
    files = os.listdir(path)
    temp_df = pd.DataFrame({'images':list(map(lambda x: np.asarray(Image.open(os.path.join(path,x)).resize((60,80)))/255.0,files)), 'dx':[img_class]*len(files)})
    test_df = pd.concat([test_df, temp_df])

In [ ]:
# val_df = pd.concat([x_val_meta, y_val_meta],axis=1)
# val_df['images'] = list(map(lambda x: np.asarray(Image.open(x).resize((90,120))),val_df['path']))

# test_df = pd.concat([x_test_meta, y_test_meta],axis=1)
# test_df['images'] = list(map(lambda x: np.asarray(Image.open(x).resize((90,120))),test_df['path']))

In [ ]:
# Checking the image size distribution
data_df['images'].map(lambda x: x.shape).value_counts()

In [ ]:
data_df['cell_type_idx'] = pd.Categorical(data_df['dx']).codes

In [ ]:
x_train = data_df['images']
y_train = data_df['cell_type_idx']

In [ ]:
y_train

## Dividing data into training and testing dataset

In [ ]:
x_val, x_test, y_val, y_test = val_df['images'], test_df['images'], pd.Categorical(val_df['dx']).codes, pd.Categorical(test_df['dx']).codes

### Label Encoding

In [ ]:
# # Perform one-hot encoding on the labels
y_train = to_categorical(y_train, num_classes = 7)
y_test = to_categorical(y_test, num_classes = 7)
y_val = to_categorical(y_val, num_classes = 7)

# x_train.sex = x_train.sex.astype('category').cat.codes
# x_train.localization = x_train.localization.astype('category').cat.codes
# x_train.dx_type = x_train.dx_type.astype('category').cat.codes

# Splitting into training and validation

Convert dataframes to numpy arrays

In [ ]:
x_train_img = np.asarray(x_train.tolist())
x_validate_img = np.asarray(x_val.tolist())
x_test_img = np.asarray(x_test.tolist())

# Model Building

In [ ]:
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

# # instantiate a distribution strategy
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)
del model
del temp_df
del path
del files
gc.collect()

## A. Building from scratch (using CNN)

In [ ]:
input_shape = (80, 60, 3)
num_classes = 7

model = keras.models.Sequential()
model.add(keras.layers.Input(shape=input_shape))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
# model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
# model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
# model.add(keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
# model.add(keras.layers.MaxPooling2D())
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
# model.add(keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal'))
# model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D())

model.add(keras.layers.Flatten())

model.add(keras.layers.Dropout(rate=0.3))
model.add(keras.layers.Dense(units=256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=keras.regularizers.L1L2() ))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=128, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.BatchNormalization())

# model.add(keras.layers.Dropout(rate=0.1))
# model.add(keras.layers.Dense(units=128, activation='relu', kernel_initializer='he_normal'))
# model.add(keras.layers.BatchNormalization())

# model.add(keras.layers.Dropout(rate=0.1))
model.add(keras.layers.Dense(units=64, activation='relu', kernel_initializer='he_normal',kernel_regularizer=keras.regularizers.L1L2()))
model.add(keras.layers.BatchNormalization())

# model.add(keras.layers.Dropout(rate=0.1))
model.add(keras.layers.Dense(units=32, activation='relu', kernel_initializer='he_normal', kernel_regularizer=keras.regularizers.L1L2()))
model.add(keras.layers.BatchNormalization())

model.add(keras.layers.Dense(units=7, activation='softmax', kernel_initializer='glorot_uniform', name='classifier'))
model.compile(Adamax(learning_rate= 0.00001), loss= 'categorical_crossentropy', metrics= ['accuracy', 'f1_score'])

model.summary()

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=3,
                                            verbose=1,
                                            factor=0.05,
                                            min_lr=0.0000001)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

### Train the model

In [ ]:
epochs = 100
batch_size = 16
history = model.fit(x_train_img, y_train
                    ,epochs = epochs
                    ,validation_data = (x_validate_img, y_val)
                    ,verbose = 1
                    , batch_size= batch_size
                    , callbacks=[learning_rate_reduction, early_stopping])
model.save('trained_model.keras')

## B. Integrating EfficientNetB2

In [ ]:
input_shape = (80, 60, 3)
inputs = layers.Input(shape=input_shape)
pre_trained_model = EfficientNetB2(include_top=False, input_tensor=inputs, weights="imagenet")

# Freeze the pretrained weights
pre_trained_model.trainable = False

# Rebuild top
x = layers.GlobalAveragePooling2D(name="avg_pool")(pre_trained_model.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.4
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

# Compile
pre_trained_model = keras.Model(inputs, outputs, name="EfficientNet")
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
pre_trained_model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", "f1_score"]
)

In [ ]:
input_shape = (80, 60, 3)
num_classes = 7
pre_trained_model = EfficientNetB2(weights=None, input_shape=input_shape, classes = num_classes)
pre_trained_model.compile(Adamax(learning_rate= 0.0001), loss= 'categorical_crossentropy', metrics= ['accuracy','f1_score'])
# pre_trained_model.summary()

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                           min_lr=0.00000001)

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

### Train the model

In [ ]:
epochs = 110
batch_size = 16
pre_trained_history = pre_trained_model.fit(x_train_img, y_train
                    ,epochs = epochs
                    ,validation_data = (x_validate_img, y_val)
                    ,verbose = 1
                    , steps_per_epoch=x_train_img.shape[0] // batch_size
                    , callbacks=[learning_rate_reduction, early_stopping])
pre_trained_model.save('pre-trained_model.keras')

## Model Evaluation

In [ ]:
def plot_model_history(hist, model_name):
    tr_acc = hist.history['accuracy']
    tr_loss = hist.history['loss']
    val_acc = hist.history['val_accuracy']
    val_loss = hist.history['val_loss']
    index_loss = np.argmin(val_loss)
    val_lowest = val_loss[index_loss]
    index_acc = np.argmax(val_acc)
    acc_highest = val_acc[index_acc]

    plt.figure(figsize= (20, 8))
    plt.style.use('fivethirtyeight')
    Epochs = [i+1 for i in range(len(tr_acc))]
    loss_label = f'best epoch= {str(index_loss + 1)}'
    acc_label = f'best epoch= {str(index_acc + 1)}'

    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
    plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
    plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
    plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
    plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout
    plt.savefig('{}.png'.format(model_name))
    plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


### Model A (from scratch) evaluation

In [ ]:
train_loss, train_accuracy = model.evaluate(x_train_img, y_train, verbose=1)[:2]
test_loss, test_accuracy = model.evaluate(x_test_img, y_test, verbose=1)[:2]
validate_loss, validate_accuracy = model.evaluate(x_validate_img, y_val, verbose=1)[:2]
print("Train: accuracy = %f  ;  loss_v = %f" % (train_accuracy, train_loss))
print("Validation: accuracy = %f  ;  loss_v = %f" % (validate_accuracy, validate_loss))
print("Test: accuracy = %f  ;  loss = %f" % (test_accuracy, test_loss))

In [ ]:
plot_model_history(history,'trained')

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(x_test_img)

# Convert predictions classes to one hot vectors
Y_pred_classes = np.argmax(Y_pred,axis = 1)

# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1)

# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
plot_confusion_matrix(confusion_mtx, classes = range(7))

In [ ]:
label_frac_error = 1 - np.diag(confusion_mtx) / np.sum(confusion_mtx, axis=1)
plt.bar(np.arange(7),label_frac_error)
plt.xlabel('True Label')
plt.ylabel('Fraction classified incorrectly')

### Model B (using EfficientNetB2) evaluation

In [ ]:
pre_trained_train_loss, pre_trained_train_accuracy = pre_trained_model.evaluate(x_train_img, y_train, verbose=1)[:2]
pre_trained_test_loss, pre_trained_test_accuracy = pre_trained_model.evaluate(x_test_img, y_test, verbose=1)[:2]
pre_trained_validate_loss, pre_trained_validate_accuracy = pre_trained_model.evaluate(x_validate_img, y_val, verbose=1)[:2]
print("Train: accuracy = %f  ;  loss_v = %f" % (pre_trained_train_accuracy, pre_trained_train_loss))
print("Validation: accuracy = %f  ;  loss_v = %f" % (pre_trained_validate_accuracy, pre_trained_validate_loss))
print("Test: accuracy = %f  ;  loss = %f" % (pre_trained_test_accuracy, pre_trained_test_loss))

In [ ]:
plot_model_history(pre_trained_history, 'pre_trained')

In [ ]:
# Predict the values from the validation dataset
pre_trained_Y_pred = pre_trained_model.predict(x_test_img)

# Convert predictions classes to one hot vectors
pre_trained_Y_pred_classes = np.argmax(pre_trained_Y_pred,axis = 1)

# Convert validation observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1)

# compute the confusion matrix
pre_trained_confusion_mtx = confusion_matrix(Y_true, pre_trained_Y_pred_classes)
plot_confusion_matrix(pre_trained_confusion_mtx, classes = range(7))

In [ ]:
pre_trained_label_frac_error = 1 - np.diag(pre_trained_confusion_mtx) / np.sum(pre_trained_confusion_mtx, axis=1)
plt.bar(np.arange(7),pre_trained_label_frac_error)
plt.xlabel('True Label')
plt.ylabel('Fraction classified incorrectly')